In [7]:
%%latex

0.5s

\begin{align*}
\chi^2 &= \sum_i (\frac{S_i-P_r^i(x,y)}{\sigma_i})^2\\
P_r^i(x,y) &\approx P_r^i(x_0,y_0)
+\frac{\partial P_r^i(x_0,y_0)}{\partial x}*(x-x_0)
+\frac{\partial P_r^i(x_0,y_0)}{\partial y}*(y-y_0)\\
&= P_0^i + P_{x,0}^i\Delta x + P_{y,0}^i\Delta y\\
\chi^2 &= \sum_i (\frac{S_i-P_0^i + P_{x,0}^i\Delta x + P_{y,0}^i\Delta y}{\sigma_i})^2\\
S_i - P_0^i &= R_0^i\\
\chi^2 &= \sum_i \frac{1}{\sigma_i^2}((R_0^i)^2 + (P_{x,0}^i\Delta x)^2 + (P_{y,0}^i\Delta y)^2 
                                      + R_0^iP_{x,0}^i\Delta x + R_0^iP_{y,0}^i\Delta y
                                      + P_{x,0}^iP_{y,0}^i\Delta x\Delta y)

\end{align*}



<IPython.core.display.Latex object>

In [11]:
%%latex

0.5t

\begin{align*}
\sum_i \frac{1}{\sigma^2}R_0^iP_{x,0}^i &= 0\\
\sum_i \frac{1}{\sigma^2}R_0^iP_{y,0}^i &= 0\\
%\sum_i \frac{1}{\sigma^2}P_{x,0}^iP_{y,0}^i\Delta x\Delta y &\approx 0\\
\chi^2 &= \sum_i \frac{1}{\sigma_i^2}((R_0^i)^2 + (P_{x,0}^i\Delta x)^2 + (P_{y,0}^i\Delta y)^2
                                      + P_{x,0}^iP_{y,0}^i\Delta x\Delta y)

\end{align*}

<IPython.core.display.Latex object>

In [42]:
%%latex

0.5u

\begin{align*}
\chi^2 &= \sum_i \frac{1}{\sigma_i^2}((R_0^i)^2 + (P_{x,0}^i\Delta x)^2 + (P_{y,0}^i\Delta y)^2
                                      + P_{x,0}^iP_{y,0}^i\Delta x\Delta y)\\
\chi^2 &= \chi_0^2 + \sum_i \frac{1}{\sigma_i^2}((P_{x,0}^i\Delta x)^2 + (P_{y,0}^i\Delta y)^2
                                      + P_{x,0}^iP_{y,0}^i\Delta x\Delta y)\\
&= A + 
\left(
\begin{array}{c}
\Delta x & \Delta y
\end{array}
\right)
* B *
\left(
\begin{array}{cc}
\Delta x \\
\Delta y
\end{array}
\right)\\

\left(
\begin{array}{c}
\Delta x & \Delta y
\end{array}
\right)

\left(
\begin{array}{cc}
a & b\\
c & d\\
\end{array}
\right) 

\left(
\begin{array}{cc}
\Delta x \\
\Delta y
\end{array}
\right)
&= (\Delta x)^2 a + \Delta x\Delta y(b+c) + (\Delta y)^2 d\\
a &= P_{x,0}^i\\
b + c &= P_{x,0}^iP_{y,0}^i\\
b = c &= \frac{1}{2}P_{x,0}^iP_{y,0}^i\\
d &= P_{y,0}^i\\

A &= \chi_0^2\\
B &= 
\left(
\begin{array}{cc}
P_{x,0}^i & \frac{1}{2}P_{x,0}^iP_{y,0}^i\\
\frac{1}{2}P_{x,0}^iP_{y,0}^i & P_{y,0}^i\\
\end{array}
\right)
\end{align*}

<IPython.core.display.Latex object>